In [2]:
from code_tokenizer import *

# code = """
# posicao.Trigger == "DEACTIVATION" || posicao.Trigger == "POWER_LOSS"
# """

code = """
a.a
if( a == 0) {
    a = 1
}
"""

# print(color_by_char_type(code))

# parsed = math_layer(code)

# def parse(code):
#     for item in code:
#         if type(item) == list and len(item) > 0:
#             if(item[0] == 'assign'):
#                 print(item[1])

# parse(parsed)
print(math_layer(code))

[['access_group', [['word', 'a'], ['word', 'a']]], ['word', 'if', ['attached_group', ['operation', '==', [['word', 'a'], ['number', '0']]]], ['attached_block', ['assign', [['word', 'a'], ['number', '1']]]]]]
